# Get CarGurus Expected Price

In [3]:
import requests
import pandas as pd
import os
import numpy as np
import bs4
import time

In [4]:
# Read in scraped car data
data_file = os.path.join(os.getcwd(),"data","all_cars.csv") 
cars = pd.read_csv(data_file)
pd.set_option("display.max_columns",None) 
cars['mileage'] = cars['mileage'].astype('Int64')
cars['year'] = cars['year'].astype('Int64')
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53897 entries, 0 to 53896
Data columns (total 26 columns):
post_date        53897 non-null object
lastpull_ts      53897 non-null int64
link             53897 non-null object
price            53897 non-null object
year             53890 non-null Int64
make             53894 non-null object
model            53893 non-null object
body             53782 non-null object
mileage          53756 non-null Int64
title_type       28793 non-null object
city             53611 non-null object
state            53611 non-null object
seller           53897 non-null object
trim             45047 non-null object
ext_color        50306 non-null object
int_color        45125 non-null object
transmission     48804 non-null object
liters           3895 non-null float64
cylinders        53400 non-null float64
fuel_type        53663 non-null object
n_doors          41296 non-null float64
ext_condition    16478 non-null object
int_condition    16444 non-null ob

# Create Zip Codes

In [5]:
# Create dictionary of zip codes for all towns to look up expected price
# Website to look up zipcodes
# http://localistica.com/usa/ut/salt%20lake%20city/zipcodes/all-zipcodes/

def get_most_populated_zip_code(city):
    try:
        r = requests.get(f'http://localistica.com/search.aspx?q={city.lower().replace(" ", "+")}')
        url = bs4.BeautifulSoup(r.text).find("a", id="ctl09_hlZipCodesCount")['href']
        return int(bs4.BeautifulSoup(requests.get(url).text).find(id="dgZipCodes").find_all("tr")[1].td.a.text)
    except:
        return None
    

# get unique cities in dataframe
all_cities = [car for car in cars.city.unique() if type(car) == str]
keyList = [x + ", " + cars[cars.city == x].iloc[0]['state'] for x in all_cities]
# look up zipcode
zip_codes = {key: get_most_populated_zip_code(key + " " + cars[cars.city == key].iloc[0]['state']) for key in all_cities}
# hard code the ones it missed
zip_codes.update({'St. Anthony': 83445, 'Provo Canyon': 84604})
# check for missing zip codes
print(len([k for k,v in zip_codes.items() if v == None]))
zip_codes

0


{'Nibley': 84321,
 'Ogden': 84201,
 'Lindon': 84042,
 'North Salt Lake': 84054,
 'Salt Lake City': 84101,
 'Twin Falls': 83301,
 'Springville': 84663,
 'Preston': 83263,
 'Murray': 84101,
 'Sandy': 84070,
 'Pocatello': 83201,
 'West Valley City': 84081,
 'Idaho Falls': 83631,
 'Plain City': 84201,
 'Pleasant Grove': 84062,
 'Magna': 84044,
 'Pleasant View': 84201,
 'Buffalo': 82834,
 'West Haven': 84201,
 'Woods Cross': 84087,
 'Filer': 83328,
 'St. George': 84770,
 'Logan': 84321,
 'Rexburg': 83440,
 'Smithfield': 84335,
 'Syracuse': 84075,
 'Midvale': 84047,
 'Hurricane': 84737,
 'Santaquin': 84655,
 'North Logan': 84321,
 'South Salt Lake': 84101,
 'Boise': 83701,
 'American Fork': 84003,
 'Vernal': 84078,
 'Washington': 84321,
 'Layton': 84040,
 'Gardena': 90247,
 'Orem': 84057,
 'Lehi': 84005,
 'Clearfield': 84015,
 'Clinton': 84015,
 'Cedar City': 84720,
 'Kanarraville': 84742,
 'Tremonton': 84337,
 'Heber City': 84032,
 'Blackfoot': 83221,
 'Richfield': 84701,
 'Tooele': 84074,


# CarGurus Pricing
Terms of Use: https://www.cargurus.com/Cars/TermsOfUse.html

Using car make, model, mileage, year, and zip code of listing to get the privatelisting value from CarGurus to compare listed prices to an expected price

In [12]:
# list of all cars in CarGurus database
all_cars = requests.get("https://www.cargurus.com/Cars/getCarPickerReferenceDataAJAX.action?showInactive=false&useInventoryService=false&quotableCarsOnly=false&localCountryCarsOnly=true&outputFormat=REACT").json()

# gets CarGuru make and model id to find price for individual make and model
def get_cargurus_maker_and_model_ids(all_cars, car_make, car_model):
    try:
        all_models = [x for x in all_cars.get('allMakerModels').get('makers') if x.get('name') == car_make][0]
        return (all_models.get('id'), [x for x in all_models.get('models') if x.get('name') == car_model][0].get('id'))
    except (IndexError, AttributeError):
        return (None, None)
    
# gets the entity id which includes the make, model, and year of car
def get_entity_id(maker_id, model_id, car_year):
    try:
        all_entities = requests.get(f"https://www.cargurus.com/Cars/getSelectedMakerModelCarsAJAX.action?showInactive=false&useInventoryService=false&quotableCarsOnly=false&localCountryCarsOnly=true&outputFormat=REACT&maker={maker_id}").json()
        model_entity_ids = [car for car in all_entities.get('models') if car.get('id') == model_id][0]
        return [ids for ids in model_entity_ids.get('cars') if ids.get('year') == car_year][0].get('id')
    except (IndexError, AttributeError):
        return None
    
# gets the estimated listing price of the car based on entity id and the mileage
def get_price(car_make, car_model, car_year, car_mileage, car_zip_code, all_cars):
    maker_id, model_id = get_cargurus_maker_and_model_ids(all_cars, car_make, car_model)
    if not model_id or pd.isna(car_mileage):
        return None
    
    entity_id = get_entity_id(maker_id, model_id, car_year)
    
    if not entity_id:
        return None 
    # data needed to request CarGurus report
    data = {
        'carDescription.radius': 75,
        'selectedEntity': entity_id,
        'carDescription.transmission': "",
        'carDescription.mileage': car_mileage,
        'carDescription.postalCode': car_zip_code,
        'carDescription.engineId': "",
        'carDescription.vin': "",
        'carDescription.vinType': "",
        'forPrivateListing': True,
        'inventoryListingId' : ""
    }
    
    res = requests.post("https://www.cargurus.com/Cars/generateReportJsonAjax.action", data=data)
    res.raise_for_status()
    try:
        return res.json().get("priceDetails").get("privateListingPrice") #private listing price from CarGurus report
    except AttributeError:
        raise Exception(res.json())


for index, row in cars.iterrows():
    if row["city"] in zip_codes.keys():
        try:
            expected_price = get_price(row["make"], row["model"], row["year"], row["mileage"], zip_codes.get(row["city"]), all_cars)
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
        # change expected prices that are 0 to none
        if expected_price == None or expected_price < 1:
            expected_price = None 
            print(index)
        cars.loc[index, "expected_price"] = expected_price
    else:
        cars.loc[index, "expected_price"] = None
        

0
3
4
7
9
11
26
27
28
31
32
33
34
35
36
37
39
41
42
45
46
48
54
56
57
58
64
65
66
68
72
73
75
78
80
82
85
88
94
97
103
105
108
113
116
118
120
121
122
125
126
128
130
132
133
134
135
136
138
139
142
144
145
146
147
148
149
151
153
155
156
157
159
160
161
162
164
166
167
172
173
175
178
180
182
185
187
188
189
190
192
193
196
197
198
200
202
206
207
209
211
213
214
215
217
218
219
222
223
227
230
231
233
242
243
248
249
251
252
255
257
262
264
265
267
276
278
282
284
285
291
294
295
297
298
300
301
306
312
313
315
316
320
324
325
326
327
328
330
331
332
334
335
336
338
339
341
343
348
349
351
358
359
361
362
364
367
369
371
372
373
374
376
379
380
381
382
383
388
389
390
391
392
393
395
397
398
399
402
404
405
410
413
417
419
421
422
426
427
428
429
430
431
434
435
437
438
439
441
444
445
447
448
452
453
455
456
458
461
462
463
464
465
466
469
470
471
475
479
480
481
482
483
484
487
488
489
492
494
495
496
497
498
500
504
506
507
508
509
514
515
520
523
524
525
527
529
532
533
536
538
5

4236
4237
4241
4242
4244
4248
4250
4251
4252
4254
4262
4263
4266
4267
4268
4269
4271
4273
4277
4280
4282
4283
4289
4290
4291
4295
4296
4298
4299
4301
4303
4305
4306
4307
4308
4310
4311
4312
4313
4314
4316
4318
4319
4321
4323
4327
4334
4335
4341
4344
4345
4346
4348
4351
4352
4354
4362
4364
4368
4371
4373
4377
4378
4379
4384
4385
4386
4387
4388
4390
4391
4392
4395
4396
4398
4400
4402
4403
4405
4409
4413
4416
4417
4422
4423
4424
4426
4430
4432
4433
4435
4437
4438
4439
4444
4445
4447
4451
4452
4454
4458
4462
4463
4464
4468
4470
4474
4480
4481
4484
4487
4488
4489
4494
4495
4496
4497
4498
4499
4502
4504
4505
4508
4510
4513
4522
4525
4526
4529
4530
4531
4533
4534
4536
4537
4539
4541
4543
4544
4545
4550
4551
4553
4556
4557
4558
4560
4561
4566
4567
4571
4572
4575
4576
4577
4579
4583
4585
4587
4589
4591
4593
4598
4600
4601
4603
4608
4609
4613
4614
4616
4617
4618
4619
4620
4625
4630
4636
4637
4639
4641
4643
4644
4648
4649
4650
4651
4656
4657
4658
4663
4664
4666
4668
4671
4676
4677
4680
4685
4686


8766
8769
8770
8772
8774
8777
8779
8780
8781
8784
8786
8787
8788
8790
8794
8796
8797
8800
8802
8803
8805
8806
8807
8813
8814
8816
8818
8819
8820
8822
8824
8825
8827
8829
8830
8831
8832
8834
8835
8837
8839
8841
8842
8843
8844
8850
8851
8852
8853
8858
8861
8862
8864
8866
8869
8873
8876
8884
8885
8887
8888
8892
8893
8895
8899
8900
8901
8906
8909
8918
8922
8923
8924
8926
8928
8930
8931
8939
8940
8941
8944
8947
8948
8949
8950
8955
8957
8958
8960
8961
8963
8966
8967
8969
8971
8972
8975
8978
8981
8982
8983
8985
8988
8989
8990
8992
8995
8996
8999
9000
9001
9003
9010
9013
9014
9015
9017
9019
9025
9027
9028
9032
9037
9039
9041
9042
9043
9044
9046
9049
9050
9056
9057
9059
9063
9064
9067
9080
9082
9086
9087
9090
9091
9092
9094
9103
9104
9105
9107
9108
9113
9116
9118
9121
9128
9131
9132
9134
9136
9138
9139
9140
9141
9144
9146
9149
9151
9154
9155
9156
9157
9160
9166
9167
9170
9173
9174
9181
9183
9185
9186
9187
9189
9196
9199
9201
9202
9205
9208
9213
9214
9217
9218
9222
9227
9228
9230
9231
9232
9233


12581
12584
12585
12586
12589
12590
12593
12595
12596
12599
12600
12604
12605
12606
12608
12609
12610
12612
12614
12615
12618
12619
12625
12629
12631
12634
12635
12639
12646
12648
12651
12652
12653
12654
12661
12664
12667
12668
12672
12678
12682
12684
12689
12690
12691
12694
12701
12706
12707
12708
12711
12715
12720
12721
12723
12725
12727
12728
12732
12740
12741
12743
12744
12745
12750
12751
12752
12754
12755
12759
12762
12763
12764
12767
12768
12769
12771
12772
12773
12774
12776
12777
12779
12782
12783
12786
12787
12788
12789
12790
12793
12794
12795
12801
12802
12806
12810
12811
12812
12813
12815
12816
12817
12818
12821
12823
12824
12828
12829
12831
12833
12836
12838
12840
12841
12842
12846
12848
12850
12852
12856
12859
12860
12862
12864
12868
12870
12871
12872
12874
12878
12881
12885
12886
12889
12892
12895
12896
12897
12898
12899
12901
12903
12907
12912
12916
12917
12918
12919
12924
12925
12928
12932
12934
12937
12943
12944
12948
12950
12951
12954
12957
12960
12961
12963
12965
1296

16314
16315
16317
16319
16320
16321
16322
16325
16328
16329
16331
16337
16340
16350
16355
16361
16362
16364
16365
16369
16378
16381
16384
16386
16387
16390
16397
16400
16403
16404
16407
16410
16414
16428
16431
16436
16438
16446
16449
16451
16453
16458
16460
16461
16466
16467
16470
16472
16473
16474
16475
16476
16477
16482
16490
16495
16497
16499
16504
16505
16508
16510
16512
16520
16522
16531
16532
16533
16537
16540
16543
16550
16552
16555
16556
16558
16559
16561
16563
16564
16565
16566
16567
16569
16573
16574
16575
16578
16582
16585
16588
16590
16591
16593
16595
16597
16598
16604
16605
16606
16607
16610
16611
16613
16614
16615
16616
16620
16622
16623
16624
16626
16628
16631
16632
16633
16634
16637
16638
16639
16641
16642
16649
16651
16652
16653
16654
16655
16658
16663
16665
16670
16671
16680
16683
16684
16685
16686
16691
16695
16697
16699
16700
16703
16705
16708
16709
16712
16714
16715
16716
16717
16718
16723
16725
16726
16727
16728
16740
16741
16742
16744
16745
16746
16754
16755
1676

20630
20633
20634
20637
20640
20641
20642
20644
20645
20647
20650
20652
20656
20659
20660
20668
20672
20676
20678
20681
20687
20693
20694
20696
20701
20703
20704
20709
20711
20714
20718
20720
20724
20731
20734
20738
20744
20748
20757
20759
20760
20763
20767
20770
20772
20773
20774
20775
20776
20777
20779
20780
20781
20784
20785
20790
20791
20794
20796
20798
20799
20802
20805
20807
20808
20809
20822
20823
20824
20826
20830
20831
20834
20835
20836
20842
20848
20851
20852
20855
20858
20859
20860
20865
20868
20871
20874
20875
20881
20892
20895
20896
20897
20898
20902
20903
20909
20911
20917
20918
20920
20925
20936
20938
20941
20942
20943
20945
20949
20960
20963
20972
20974
20975
20979
20981
20983
20988
20993
20995
20996
20997
20999
21000
21002
21003
21007
21008
21012
21014
21016
21019
21022
21025
21027
21028
21032
21033
21035
21037
21038
21039
21041
21042
21044
21046
21047
21050
21052
21056
21064
21065
21069
21070
21072
21074
21079
21080
21082
21086
21088
21092
21102
21105
21108
21109
2111

25353
25354
25355
25357
25359
25360
25362
25364
25365
25367
25368
25369
25370
25371
25374
25375
25378
25379
25380
25385
25386
25392
25393
25394
25395
25396
25399
25400
25403
25406
25411
25412
25413
25415
25416
25418
25419
25420
25422
25424
25427
25428
25429
25430
25431
25433
25435
25438
25442
25444
25445
25453
25454
25456
25463
25465
25466
25468
25471
25473
25474
25478
25480
25481
25482
25484
25485
25488
25499
25500
25501
25504
25511
25512
25513
25514
25517
25518
25519
25522
25524
25526
25528
25529
25532
25540
25542
25544
25556
25559
25561
25562
25569
25573
25577
25578
25579
25585
25586
25588
25589
25592
25596
25598
25600
25605
25608
25609
25617
25618
25619
25622
25623
25626
25629
25631
25632
25634
25635
25636
25641
25645
25648
25649
25651
25652
25654
25655
25658
25660
25662
25663
25665
25668
25677
25678
25681
25684
25691
25692
25693
25701
25708
25710
25718
25719
25722
25723
25732
25740
25743
25754
25755
25761
25766
25768
25779
25780
25784
25801
25802
25805
25808
25810
25812
25817
2581

30027
30028
30031
30033
30034
30035
30037
30039
30040
30043
30045
30049
30051
30052
30053
30056
30057
30058
30061
30063
30065
30069
30071
30076
30081
30086
30089
30091
30093
30094
30097
30098
30101
30106
30107
30108
30109
30110
30111
30112
30114
30117
30118
30119
30128
30130
30131
30132
30133
30137
30141
30143
30147
30150
30151
30152
30155
30156
30157
30158
30160
30164
30169
30171
30172
30173
30175
30178
30179
30182
30184
30196
30198
30199
30201
30202
30207
30209
30210
30211
30219
30221
30222
30232
30233
30234
30235
30236
30238
30239
30242
30245
30246
30247
30248
30249
30252
30253
30257
30263
30266
30267
30272
30274
30276
30281
30288
30289
30290
30291
30292
30293
30295
30302
30304
30307
30314
30315
30317
30318
30319
30321
30326
30327
30328
30334
30339
30343
30345
30346
30349
30350
30355
30360
30365
30368
30372
30374
30376
30377
30378
30380
30381
30382
30383
30387
30389
30391
30392
30394
30395
30401
30407
30408
30411
30412
30418
30423
30425
30426
30431
30436
30440
30444
30452
30456
3045

34394
34395
34396
34398
34416
34419
34420
34421
34426
34428
34430
34431
34434
34435
34436
34440
34441
34442
34443
34444
34445
34449
34451
34452
34456
34457
34459
34461
34462
34464
34467
34471
34475
34483
34491
34497
34504
34506
34507
34509
34510
34511
34512
34517
34522
34528
34531
34534
34535
34536
34537
34538
34539
34542
34543
34544
34546
34548
34549
34551
34554
34556
34558
34561
34563
34565
34568
34569
34570
34571
34573
34575
34580
34582
34583
34584
34593
34595
34596
34602
34604
34606
34612
34616
34623
34624
34625
34626
34628
34629
34630
34634
34635
34638
34639
34640
34643
34648
34649
34651
34652
34653
34655
34657
34659
34660
34664
34669
34670
34681
34683
34687
34689
34690
34691
34693
34697
34700
34703
34707
34708
34710
34711
34713
34716
34718
34719
34720
34722
34724
34725
34728
34734
34735
34740
34741
34742
34746
34748
34750
34755
34757
34758
34760
34766
34767
34769
34771
34772
34773
34774
34775
34776
34779
34780
34782
34783
34785
34786
34788
34791
34794
34800
34802
34803
34812
3481

In [19]:
# Add zip codes to a column
for index, row in cars.iterrows():
    cars.loc[index, "zip_code"] = zip_codes.get(row["city"])


In [25]:
cars.head(5315)

# save cars dataframe to pickle
cars.to_pickle("./cars.pkl")

In [24]:
# Check read out worked
pickle_cars = pd.read_pickle('cars.pkl')
pickle_cars.info() # 24,315 expected prices

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53897 entries, 0 to 53896
Data columns (total 28 columns):
post_date         53897 non-null object
lastpull_ts       53897 non-null int64
link              53897 non-null object
price             53897 non-null object
year              53890 non-null Int64
make              53894 non-null object
model             53893 non-null object
body              53782 non-null object
mileage           53756 non-null Int64
title_type        28793 non-null object
city              53611 non-null object
state             53611 non-null object
seller            53897 non-null object
trim              45047 non-null object
ext_color         50306 non-null object
int_color         45125 non-null object
transmission      48804 non-null object
liters            3895 non-null float64
cylinders         53400 non-null float64
fuel_type         53663 non-null object
n_doors           41296 non-null float64
ext_condition     16478 non-null object
int_conditio

In [15]:
display(pickle_cars)

,post_date,lastpull_ts,link,price,year,make,model,body,mileage,title_type,city,state,seller,trim,ext_color,int_color,transmission,liters,cylinders,fuel_type,n_doors,ext_condition,int_condition,drive_type,VIN,n_pics,expected_price
0,2020-04-05,1586150574,https://cars.ksl.com/listing/6352088,1200,1968,Oldsmobile,Toronado,Coupe,99999,Clean Title,Nibley,UT,Owner,NaN,gold,black,Automatic,NaN,8.0,Gasoline,2.0,Fair,Fair,FWD,XXXXXXXXX,8,NaN
1,2020-04-05,1586150578,https://cars.ksl.com/listing/6320528,14823,2019,Chevrolet,Spark,Hatchback,7609,NaN,Ogden,UT,Dealer,LT,black,jet black/dark anderson silver metallic,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,FWD,KL8CD6SAXKC805721,26,10220.0
2,2020-04-05,1586150578,https://cars.ksl.com/listing/6328730,21997,2018,Hyundai,Tucson,Sport Utility,15311,NaN,Lindon,UT,Dealer,SEL Plus,silver,black,Automatic,NaN,4.0,Gasoline,4.0,NaN,NaN,AWD,KM8J3CA44JU813314,0,16554.0
3,2020-04-05,1586150579,https://cars.ksl.com/listing/6293848,8500,1993,Ford,Bronco,NaN,115871,NaN,North Salt Lake,UT,Dealer,XLT,white,gray,Automatic,NaN,8.0,Gasoline,NaN,NaN,NaN,4-Wheel Drive,1FMEU15H0PLA54840,20,NaN
4,2020-04-05,1586150579,https://cars.ksl.com/listing/6082281,12990,2013,Ram,2500,Truck,107111,Clean Title,Salt Lake City,UT,Dealer,Tradesman,white,black,Automatic,NaN,8.0,Gasoline,4.0,Excellent,Excellent,4-Wheel Drive,3C6TR5HT6DG587537,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53892,2020-03-20,1584833057,https://cars.ksl.com/listing/6269151,20148,2019,Ford,Fusion Hybrid,Sedan,20303,NaN,Price,UT,Dealer,SE,silver,NaN,NaN,NaN,4.0,Hybrid,NaN,NaN,NaN,FWD,3FA6P0LU0KR206951,0,15072.0
53893,2020-03-20,1584833058,https://cars.ksl.com/listing/5968149,15995,2009,Jeep,Grand Cherokee,Sport Utility,122000,NaN,NaN,NaN,Dealer,SRT-8,red,charcoal gray,Automatic,NaN,8.0,Gasoline,4.0,NaN,NaN,4-Wheel Drive,1J8HR78W89C505718,19,NaN
53894,2020-03-20,1584833058,https://cars.ksl.com/listing/6287241,52900,2020,Acura,MDX,Sport Utility,2080,NaN,Salt Lake City,UT,Dealer,with Advance/Entertainment Package,white,black,Automatic,NaN,6.0,Gasoline,NaN,NaN,NaN,AWD,5J8YD4H99LL011505,50,37806.0
53895,2020-03-20,1584833059,https://cars.ksl.com/listing/6269149,22325,2019,Hyundai,Santa Fe,Sport Utility,17195,NaN,Price,UT,Dealer,SE,quartz white,espresso/gray,Automatic,NaN,4.0,Gasoline,NaN,NaN,NaN,AWD,5NMS2CAD0KH106182,21,20292.0
